In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.tools import tool
import requests
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain_community.tools import DuckDuckGoSearchRun
import os

In [2]:
load_dotenv()

True

In [3]:
weather_api_key = os.getenv("WEATHER_STACK_API_KEY")

In [4]:
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [5]:
llm.invoke('Hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--bee7382c-7fa3-4a1a-babd-5127a97f8265-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [6]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt

/Users/daniyalkhan/anaconda3/envs/genAI/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [8]:
search_tool = DuckDuckGoSearchRun()

@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key={weather_api_key}&query={city}'

  response = requests.get(url)

  return response.json()

In [9]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [10]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [11]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh, then find it's current weather condition"})
print(response)



> Entering new AgentExecutor chain...
I need to first find the capital of Madhya Pradesh, and then use that information to find the current weather condition.
Action: duckduckgo_search
Action Input: "capital of Madhya Pradesh"

/Users/daniyalkhan/anaconda3/envs/genAI/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


No good DuckDuckGo Search Result was foundI apologize, I seem to be having trouble with the search. I will try a different approach.
Action: duckduckgo_search
Action Input: what is the capital of Madhya Pradesh?

/Users/daniyalkhan/anaconda3/envs/genAI/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Bhopal is the capital of Madhya Pradesh. Bhopal is the administrative headquarters. This article will shed more light on the capital city of Bhopal and the state of Madhya Pradesh. Bhopal … The capital city of the state Madhya Pradesh is Bhopal. The city is known for its various natural and artificial lakes and for being one of the greenest cities in India. Which is India’s first Justice City? Andhra Pradesh is building a world-class, India’s first Justice city within its capital Amaravathi with a view to providing a state of art ecosystem built on new … Madhya Pradesh is the largest spice-producing state in India. India is the world’s largest producer of spices. India is also the world’s largest exporter of spices. This article will highlight the … The capital city of the state Uttar Pradesh is Lucknow. Uttar Pradesh is the highly populated state in India. The population of this state is around 19.96 crores. About Uttar Pradesh Uttar …I now know the capital of Madhya Pradesh is Bhopal

In [12]:
response['output']

'The capital of Madhya Pradesh is Bhopal and the current weather condition is Rain with a temperature of 23 degrees Celsius.'